In [5]:
import subprocess, sys, shutil
from pathlib import Path

# --- 1) Locate or auto-install an ffmpeg binary we can call ---
def get_ffmpeg_path():
    # a) Already on PATH?
    p = shutil.which("ffmpeg")
    if p:
        return Path(p)

    # b) Try imageio-ffmpeg (downloads a local static ffmpeg if missing)
    try:
        import imageio_ffmpeg
    except ModuleNotFoundError:
        print("Installing imageio-ffmpeg (one-time)…")
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", "imageio-ffmpeg"], check=True)
        import imageio_ffmpeg

    return Path(imageio_ffmpeg.get_ffmpeg_exe())

ffmpeg_path = get_ffmpeg_path()

# Optional: show version (helps confirm it's working)
try:
    out = subprocess.check_output([str(ffmpeg_path), "-version"], text=True, stderr=subprocess.STDOUT)
    print(out.splitlines()[0])
except Exception as e:
    raise RuntimeError(f"ffmpeg exists at {ffmpeg_path} but couldn't run it: {e}")

# --- 2) Batch convert settings ---
input_folder  = Path("Videos")
output_folder = Path("videos_transformed")
output_folder.mkdir(exist_ok=True)

CRF    = "23"       # 18–24 common range; lower = better quality, larger files
PRESET = "medium"   # ultrafast..placebo (slower = smaller file)
FPS    = "20"

videos = sorted(input_folder.glob("*.mp4"))
if not videos:
    print("⚠️ No .mp4 files found in 'Videos'. Make sure the folder sits next to this notebook.")
else:
    for src in videos:
        dst = output_folder / f"{src.stem}_720p20fps.mp4"
        print(f"Processing: {src.name}")

        # Fit inside 720p without stretching; keep mp4 (H.264 + AAC), 20 fps
        cmd = [
            str(ffmpeg_path),
            "-y",
            "-i", str(src),
            "-vf", f"scale=-2:720:force_original_aspect_ratio=decrease,fps={FPS}",
            "-r", FPS,
            "-c:v", "libx264",
            "-preset", PRESET,
            "-crf", CRF,
            "-pix_fmt", "yuv420p",
            "-c:a", "aac",
            "-b:a", "160k",
            str(dst)
        ]

        # Run and stream output so you can see progress in Jupyter
        proc = subprocess.run(cmd, text=True)
        if proc.returncode == 0:
            print(f"✅ Saved: {dst.name}\n")
        else:
            print(f"❌ Failed: {src.name}\n")

    print("🎬 Done.")


ffmpeg version 7.1-essentials_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
Processing: 20251122_203332.mp4
✅ Saved: 20251122_203332_720p20fps.mp4

Processing: 20251122_203353.mp4
✅ Saved: 20251122_203353_720p20fps.mp4

Processing: 20251122_203414.mp4
✅ Saved: 20251122_203414_720p20fps.mp4

Processing: 20251122_203440.mp4
✅ Saved: 20251122_203440_720p20fps.mp4

Processing: 20251122_203453.mp4
✅ Saved: 20251122_203453_720p20fps.mp4

Processing: 20251122_203611.mp4
✅ Saved: 20251122_203611_720p20fps.mp4

Processing: 20251122_203623.mp4
✅ Saved: 20251122_203623_720p20fps.mp4

Processing: 20251122_203638.mp4
✅ Saved: 20251122_203638_720p20fps.mp4

Processing: 20251122_203755.mp4
✅ Saved: 20251122_203755_720p20fps.mp4

Processing: 20251122_203800.mp4
✅ Saved: 20251122_203800_720p20fps.mp4

Processing: 20251122_203811.mp4
✅ Saved: 20251122_203811_720p20fps.mp4

Processing: 20251122_203853.mp4
✅ Saved: 20251122_203853_720p20fps.mp4

Processing: 20251122_203859.mp4
✅ Saved: 

In [6]:
import subprocess
from pathlib import Path
import shutil

# === Locate ffmpeg again (reuse imageio-ffmpeg if needed) ===
def get_ffmpeg_path():
    p = shutil.which("ffmpeg")
    if p:
        return Path(p)
    import imageio_ffmpeg
    return Path(imageio_ffmpeg.get_ffmpeg_exe())

ffmpeg_path = get_ffmpeg_path()

# === Folders ===
input_folder = Path("videos_transformed")
output_folder = Path("videos_noaudio")
output_folder.mkdir(exist_ok=True)

videos = sorted(input_folder.glob("*.mp4"))

if not videos:
    print("⚠️ No .mp4 videos found in 'videos_transformed'.")
else:
    for src in videos:
        dst = output_folder / f"{src.stem}_nosound.mp4"
        print(f"Removing sound from: {src.name}")

        cmd = [
            str(ffmpeg_path),
            "-y",
            "-i", str(src),
            "-an",          # removes all audio streams
            "-c:v", "copy", # keep video stream unchanged
            str(dst)
        ]

        subprocess.run(cmd, check=True)
        print(f"✅ Saved silent video: {dst.name}\n")

    print("🎬 All videos processed successfully — no audio now!")


Removing sound from: 20251122_203332_720p20fps.mp4
✅ Saved silent video: 20251122_203332_720p20fps_nosound.mp4

Removing sound from: 20251122_203353_720p20fps.mp4
✅ Saved silent video: 20251122_203353_720p20fps_nosound.mp4

Removing sound from: 20251122_203414_720p20fps.mp4
✅ Saved silent video: 20251122_203414_720p20fps_nosound.mp4

Removing sound from: 20251122_203440_720p20fps.mp4
✅ Saved silent video: 20251122_203440_720p20fps_nosound.mp4

Removing sound from: 20251122_203453_720p20fps.mp4
✅ Saved silent video: 20251122_203453_720p20fps_nosound.mp4

Removing sound from: 20251122_203611_720p20fps.mp4
✅ Saved silent video: 20251122_203611_720p20fps_nosound.mp4

Removing sound from: 20251122_203623_720p20fps.mp4
✅ Saved silent video: 20251122_203623_720p20fps_nosound.mp4

Removing sound from: 20251122_203638_720p20fps.mp4
✅ Saved silent video: 20251122_203638_720p20fps_nosound.mp4

Removing sound from: 20251122_203755_720p20fps.mp4
✅ Saved silent video: 20251122_203755_720p20fps_nosou

In [7]:
import subprocess, shutil
from pathlib import Path

# -------- Locate ffmpeg (PATH or imageio-ffmpeg fallback) --------
def get_ffmpeg_path():
    p = shutil.which("ffmpeg")
    if p:
        return Path(p)
    import imageio_ffmpeg
    return Path(imageio_ffmpeg.get_ffmpeg_exe())

ffmpeg = get_ffmpeg_path()

# -------- Paths & categories --------
base = Path("videos_noaudio")
categories = ["Rock", "Paper", "scissor"]  # use your exact folder names

# -------- Extraction settings --------
# Output image format: jpg (you can switch to png if you prefer)
# JPEG quality: -q:v 2 is high quality (2 best .. 31 worst)
jpg_quality = "2"

any_found = False
for cat in categories:
    folder = base / cat
    if not folder.exists():
        print(f"⚠️ Folder not found: {folder}")
        continue

    videos = sorted(folder.glob("*.mp4"))
    if not videos:
        print(f"⚠️ No .mp4 files in {folder}")
        continue

    any_found = True
    for src in videos:
        # Frame file pattern: <video_stem>_001.jpg, <video_stem>_002.jpg, ...
        out_pattern = str(folder / f"{src.stem}_%03d.jpg")
        print(f"Extracting frames from: {src.relative_to(base)}")

        cmd = [
            str(ffmpeg),
            "-y",
            "-i", str(src),
            "-start_number", "1",
            "-q:v", jpg_quality,   # JPEG quality (lower is better)
            out_pattern
        ]

        # Run ffmpeg; this extracts every frame at the video's native fps
        proc = subprocess.run(cmd)
        if proc.returncode == 0:
            print(f"✅ Frames saved as: {src.stem}_001.jpg, {src.stem}_002.jpg, ... in {folder}\n")
        else:
            print(f"❌ Failed on {src}\n")

if not any_found:
    print("No videos found to process. Check your folder names and contents.")
else:
    print("🎬 Done extracting frames.")


Extracting frames from: Rock\20251122_215015_720p20fps_nosound.mp4
✅ Frames saved as: 20251122_215015_720p20fps_nosound_001.jpg, 20251122_215015_720p20fps_nosound_002.jpg, ... in videos_noaudio\Rock

Extracting frames from: Rock\20251122_215024_720p20fps_nosound.mp4
✅ Frames saved as: 20251122_215024_720p20fps_nosound_001.jpg, 20251122_215024_720p20fps_nosound_002.jpg, ... in videos_noaudio\Rock

Extracting frames from: Rock\20251122_220443_720p20fps_nosound.mp4
✅ Frames saved as: 20251122_220443_720p20fps_nosound_001.jpg, 20251122_220443_720p20fps_nosound_002.jpg, ... in videos_noaudio\Rock

Extracting frames from: Rock\20251122_220455_720p20fps_nosound.mp4
✅ Frames saved as: 20251122_220455_720p20fps_nosound_001.jpg, 20251122_220455_720p20fps_nosound_002.jpg, ... in videos_noaudio\Rock

Extracting frames from: Rock\20251122_220502_720p20fps_nosound.mp4
✅ Frames saved as: 20251122_220502_720p20fps_nosound_001.jpg, 20251122_220502_720p20fps_nosound_002.jpg, ... in videos_noaudio\Rock

